# Electrical Power Generation (Base)

A set of generators is available to satisfy power demand for the following day.  Anticipated demand is as follows:

| Time Period | Demand (megawatts) |
| --- | --- |
| 12 am to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 am | 27000 |

Generators are grouped into three types, with the following minimum and maximum output for each type (when they are on):

| Type | Number available | Minimum output (MW) | Maximum output (MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

There are costs associated with using a generator: a cost per hour when the generator is on (and generating its minimum output), a cost per megawatt hour above its minimum, and a startup cost for turning a generator on:

| Type | Cost per hour (when on) | Cost per MWh above minimum | Startup cost |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.  For this model, the set of selected generators must be able to produce as much as 115% of predicted demand.

Which generators should be committed to meeting anticipated demand in order to minimize total cost?

---
## Model Formulation

### Sets and Indices

$t \in \text{Types}=\{0,1,2\}$: Types of generators.

$p \in \text{Periods}=\{0,1,2,3,4\}$: Time periods.

### Parameters

$\text{period-hours}_p \in \mathbb{N}^+$: Number of hours in each time period.

$\text{generators}_t \in \mathbb{N}^+$: Number of generators of type $t$.

$\text{demand}_p \in \mathbb{R}^+$: Total power demand for time period $p$.

$\text{start0} \in \mathbb{N}^+$: Number of generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).

$\text{min-output}_t \in \mathbb{R}^+$: Minimum output for generator type $t$ (when on).

$\text{max-output}_t \in \mathbb{R}^+$: Maximum output for generator type $t$.

$\text{base-cost}_t \in \mathbb{R}^+$: Minimum operating cost (per hour) for a generator of type $t$.

$\text{per-mwh-cost}_t \in \mathbb{R}^+$: Cost to generate one additional MW (per hour) for a generator of type $t$.

$\text{startup-cost}_t \in \mathbb{R}^+$: Startup cost for generator of type $t$.

### Decision Variables

$\text{ngen}_{t,p} \in \mathbb{N}^+$: Number of generators of type $t$ that are on in time period $p$.

$\text{output}_{t,p} \in \mathbb{R}^+$: Total power output from generators of type $t$ in time period $p$.

$\text{nstart}_{t,p} \in \mathbb{N}^+$: Number of generators of type $t$ to start in time period $p$.



### Objective Function

- **Cost**: Minimize the cost (in USD) to satisfy the predicted electricity demand.

\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup}
\end{equation}

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base-cost}_t*\text{period-hours}_p*\text{ngen}_{t,p}}
\end{equation}

\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per-mwh-cost}_t*\text{period-hours}_p*(\text{output}_{t,p} - \text{min-load}_t})
\end{equation}

\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup-cost}_t*\text{nstart}_{t,p}}
\end{equation}


### Constraints

- **Available generators**: Number of generators used must be less than or equal to the number available.

\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_{t} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Demand**: Total power generated across all generator types must meet anticipated demand for each time period $p$.

\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} \geq \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Min/max generation**: Power generation must respect generator min/max values.

\begin{equation}
\text{output}_{t,p} \geq \text{min-output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

\begin{equation}
\text{output}_{t,p} \leq \text{max-output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Reserve**: Selected generators must be able to satisfy demand that is as much as 15% above the prediction.

\begin{equation}
\sum_{t \in \text{Types}}{\text{max-output}_t*\text{ngen}_{t,p}} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Startup**: Establish relationship between number of active generators and number of startups (use $start0$ for period before the time horizon starts)

\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{startup}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}